# Word sense disambiguation (WSD)
It is an open problem of  natural language processing, which comprises the process of identifying which sense of a word (i.e. meaning) is used in any given sentence, when the word has a number of distinct senses (polysemy).

This part of the lab requires to implement the Lesk algorithm and evaluate the accuracy of the system.

**Type of features**: bag-of-words features.



In [3]:
import nltk 
import re
import math
import random
import csv
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import Synset
nltk.download('semcor')



[nltk_data] Downloading package semcor to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package semcor is already up-to-date!


True

Stop words are commonly used words in a language that are often considered insignificant for text analysis and information retrieval tasks.

Examples of stop words in English include "the," "is," "and," "of," and so on. These words typically have little or no semantic meaning and are frequently used in the language.

I am going to consider the implication of removing/keeping function words (e.i. stopwords ) when applying the Lesk Algorithm to the WSD Task.


In [4]:
stopwords_file = open(f"./utils/stop_words_FULL.txt", "r")
function_words_list = []
for word in stopwords_file:
    function_words_list.append(word.replace('\n', ''))
stopwords_file.close()

# Remove duplicates if any
function_words = list(set(function_words_list))
print(len(function_words), function_words[:10])

671 ['wheres', 'stop', 'less', 'wants', 'whim', 'enough', 'they', 'sufficiently', 'www', 'p']


Let's now define some utils function for handling text pre processing such as:
- punctuation removal
- function words removal ( which are a super set of the stopwords)

In [5]:
def remove_punctuation(text):
    pattern = r"[^\w\s]"  # Matches any character that is not a word character or whitespace
    cleaned_text = re.sub(pattern, "", text)
    return cleaned_text.strip()

def remove_stopwords(sentence: str):
    words = sentence.split()
    cleaned_words = []
    for word in words:
        if word not in function_words:
            cleaned_words.append(word)
    return cleaned_words

example_sentence = "The quick brown fox jumps over the lazy dog. Is it true?".lower()
print(remove_stopwords(example_sentence))

print(remove_punctuation(example_sentence))



['quick', 'brown', 'fox', 'jumps', 'lazy', 'dog.', 'true?']
the quick brown fox jumps over the lazy dog is it true


In the context of Word Sense Disambiguation (WSD), the Bag-of-Words (BoW) approach is a common representation technique used to transform textual data into a numerical format that can be processed by machine learning algorithms.

The BoW approach treats each document or sentence as an unordered set of words, disregarding the grammar and word order, and focuses solely on the frequency or presence of words. 

The following implementation considers the simplest bow rappresentation:  the context of a target word is rappresented by a vector of features, each binary feature indicating whether a vocabulary word w does or doesn’t occur in the context

In [6]:

def build_bag_of_words(phrase: str, cleanse=True) -> list:
    """
    Returns the set of words of a phrase.
    """
    sentence = phrase.lower()
    words = []
    sentence: str = remove_punctuation(sentence)
    if cleanse:
        words = remove_stopwords(sentence)
    else:
        words = sentence.split()
    return words

example_sentence = "The quick brown fox jumps over the lazy dog. Is it true?"

print(build_bag_of_words(example_sentence,cleanse=True))


['quick', 'brown', 'fox', 'jumps', 'lazy', 'dog', 'true']


The Lesk algorithm follows these main steps:

- Retrieve the context: Gather the surrounding words or the context in which the target word appears. Typically, a window of neighboring words is considered.

- Obtain the senses: Retrieve the different senses of the target word from a sense inventory or a lexical resource such as WordNet. Each sense corresponds to a different meaning or sense of the word.

- Extract glosses: Retrieve the gloss (definition) for each sense of the target word. The gloss provides a concise description of the meaning of the sense.

- Calculate overlap: Compare the words in the context with the words in each sense's gloss. Calculate the overlap or similarity between the context and the gloss by measuring the shared words or their relatedness.

- Select the best sense: Determine the sense with the highest overlap or similarity score as the predicted sense for the target word. The assumption is that the sense with the most related words in common with the context is the most likely sense.



In [7]:

def compute_overlap(signature: list, context: list) -> int:
    """
    Returns the number of words in common between signature and context
    """
    total_common_words = 0
    unique_words_signature = (set(signature))
    unique_words_context = (set(context))
    # intersaction between the two sets
    common_words = unique_words_signature.intersection(unique_words_context)
    total_common_words = len(common_words)
    return total_common_words,common_words


def get_signature(synset: Synset,cleanse) -> list:
    """
    Returns the signature of synset -> set of words in the gloss and examples of sens
    """
    examples = synset.examples() #examples of given synset
    glossary = synset.definition() #glossary of given synset

    # given examples list and the glossary pharse, we build the signature
    phrases = []
    phrases.append(glossary)
    phrases.extend(examples)
    # each phrase is converted in a bag of words
    signature = []
    for phrase in phrases:
        set_of_words = build_bag_of_words(phrase,cleanse)
        signature.extend(set_of_words)
    # remove duplicates
    signature = list(set(signature))
    return signature


def lesk(word: str, sentence: str, cleanse = True) -> Synset:
    """
    Returns the best sense of given word used in sentence.

    Args
        word: word to disambiguate
        sentence: sentence where word appears

    Returns
        best_sense: best sense of word used in sentence
    """ 
    signature_best_sense = None # signature of best sense of word used in sentence
    common_words_best_sense = None # common words between signature and context of best sense of word used in sentence

    best_sense = None # best sense of word used in sentence
    
    max_overlap = 0  # max overlap between signature and context
   
    # the context is rappresented by the bag of words of the sentence
    context = build_bag_of_words(sentence,cleanse) 
    # for each sense of the word, we compute the overlap between the signature and the context
    for synset in wn.synsets(word):
        signature = get_signature(synset,cleanse) 
        overlap,common_words = compute_overlap(signature, context)
        # print(synset,signature,context,overlap,common_words)

        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = synset
            signature_best_sense = signature
            common_words_best_sense = common_words
    if(max_overlap> 0 ):
        return best_sense, max_overlap, signature_best_sense,common_words_best_sense
    return None,None,None,None


In [83]:
# test the lesk algorithm
sentence = '''the bank can guarantee deposits will eventually cover future tuition
costs because it invests in adjustable-rate mortgage securities.'''
word = "bank"
best_sense, max_overlap, signature_best_sense, common_words_best_sense = lesk(word, sentence, cleanse=True)
print(best_sense)
if(not best_sense):
    print("No sense found for the word: ",word)
else:
     print( best_sense,
      '\n', best_sense.definition(),
      '\n', best_sense.examples(),
      '\n', max_overlap,
      '\n', signature_best_sense,
      '\n', common_words_best_sense)


Synset('depository_financial_institution.n.01')
Synset('depository_financial_institution.n.01') 
 a financial institution that accepts deposits and channels the money into lending activities 
 ['he cashed a check at the bank', 'that bank holds the mortgage on my home'] 
 3 
 ['holds', 'accepts', 'channels', 'cashed', 'check', 'financial', 'money', 'activities', 'lending', 'deposits', 'bank', 'mortgage', 'institution'] 
 {'mortgage', 'bank', 'deposits'}


In [85]:
# test the lesk algorithm
sentence = '''the bank can guarantee deposits will eventually cover future tuition
costs because it invests in adjustable-rate mortgage securities. '''
word = "bank"
best_sense, max_overlap, signature_best_sense, common_words_best_sense = lesk(word, sentence,cleanse=False)
if(not best_sense):
    print("No sense found for the word: ",word)
else:
     print( best_sense,
      '\n', best_sense.definition(),
      '\n', best_sense.examples(),
      '\n', max_overlap,
      '\n', signature_best_sense,
      '\n', common_words_best_sense)



Synset('depository_financial_institution.n.01') 
 a financial institution that accepts deposits and channels the money into lending activities 
 ['he cashed a check at the bank', 'that bank holds the mortgage on my home'] 
 4 
 ['home', 'accepts', 'on', 'financial', 'cashed', 'bank', 'institution', 'he', 'holds', 'that', 'at', 'activities', 'the', 'my', 'mortgage', 'into', 'a', 'and', 'lending', 'channels', 'check', 'money', 'deposits'] 
 {'mortgage', 'bank', 'the', 'deposits'}


In this case, the best sense is 'depository_financial_institution.m.01' but the fact that we are considering function words could easily lead to wrong interpretations. 

Let's now test the algorithm on the SemCor Corpus:
- Extract 50 sentences from the SemCor corpus (corpus annotated with the synsets of 
WN) and disambiguate (at least) one noun per sentence. Calculate 
the accuracy of the implemented system based on the senses annotated in 
SemCor.


In [11]:
from nltk.corpus import semcor

# extract 1 sentence from semcor corpus
semcor_sents = semcor.sents()
semcor_sent = semcor_sents[0]
print(semcor_sent)



['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', 'Atlanta', "'s", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']


In [74]:
import random

def get_sentences_from_semcor(randomExtract=False):
    sentences = []
    all_sentences = semcor.sents()

    if randomExtract:
        selected_sentences = random.sample(list(all_sentences[:1000]), 50)
    else:
        selected_sentences = all_sentences[:50]

    for i, sentence in enumerate(selected_sentences):
        tagged_sentence = semcor.tagged_sents(tag='sem')[i]
        sentences.append((sentence, tagged_sentence))

    return sentences





def get_nouns_from_sentence(sentenceData, randomExtract=False):
    nouns = []
    
    for targetSense in sentenceData[1]:
        if isinstance(targetSense, nltk.tree.Tree):
            lemma = targetSense.label()
            if lemma:
                synset = None
                if isinstance(lemma, str):
                    synset = wn.synsets(lemma)
                else:
                    synset = lemma.synset()
                if synset and isinstance(synset, Synset) and synset.pos() == 'n':
                    for word in targetSense.leaves():
                            nouns.append((word, synset))
    if(randomExtract):
        max_no_of_nouns = random.randint(1, len(nouns))
        nouns = random.sample(nouns, max_no_of_nouns)

    return nouns



# disambiuate the nouns in the sentence
def disambiguate_nouns(sentenceData,nounsData,cleanse=False,debug=False):
    best_senses =[]
    for nounData in nounsData:
        noun = nounData[0]
        sentence_str = ' '.join(sentenceData[0])
        best_sense, max_overlap, signature_best_sense, common_words_best_sense = lesk(noun, sentence_str,cleanse=cleanse)
        if(not best_sense):
            if(debug):
                print("No sense found for the word: ",noun)
            best_senses.append(None)
        else:
            best_senses.append(best_sense)
            if(debug):
                print("Sense for word: ",noun,
                '\n', best_sense,
                '\n', best_sense.definition(),
                '\n', best_sense.examples(),
                '\n', max_overlap,
                '\n', signature_best_sense,
                '\n', common_words_best_sense)
    return best_senses








Let's evaluate the accuracy of the implemented Lesk Algo over 50 sample sentences (the first 50).

In the first case, we will set the cleanse param to False, so function words and stop words won't be removed.
In the second iteration, data cleanse will be turned on.

In [79]:
def runTest(cleanse=False,randomExtract=False,debug=False):
    correct_senses = [0] * 50
    total_senses   = [0] * 50
    sentences = get_sentences_from_semcor(randomExtract=randomExtract)
    for i,sentenceData in enumerate(sentences):
        nouns = get_nouns_from_sentence(sentenceData,randomExtract=randomExtract)
        best_senses= disambiguate_nouns(sentenceData,nouns,cleanse=cleanse,debug=debug)

        if(best_senses):
            # https://www.nltk.org/api/nltk.corpus.reader.SemcorCorpusReader.html?highlight=tagged_sent#nltk.corpus.reader.SemcorCorpusReader.tagged_sents
            # write the code to check if the sense of the word is the same of the one in semcor

            for (nounData,best_sense) in zip(nouns,best_senses):
                # check if best_senses[ix].name() exists in sentenceData[1] Lemmas, and if it does, check if the leaf contains the noun
                if(best_sense):
                    if(best_sense == nounData[1]):
                        correct_senses [i] += 1
                    total_senses [i] += 1
    return (correct_senses, total_senses)
print("\n[with stopwords]")
correct_senses, total_senses= runTest(cleanse=False,randomExtract=False,debug=False)
debug = False
# evaluate the accuracy of the algorithm in respect to each sentence
if(debug):
    for i in range(50):
        if(total_senses[i] > 0):
            print("Accuracy for sentence ",i,": ",correct_senses[i]/total_senses[i])
        else:
            print("Accuracy for sentence ",i,": ",0)
# evaluate the accuracy of the algorithm in respect to all the sentences
print("Correct senses: ",sum(correct_senses))
print("Total senses: ",sum(total_senses))
print("Accuracy for all sentences: ",sum(correct_senses)/sum(total_senses))


correct_senses, total_senses= runTest(cleanse=True,randomExtract=False,debug=False)
print("\n[no stopwords]")
# evaluate the accuracy of the algorithm in respect to each sentence
if(debug):
    for i in range(50):
        if(total_senses[i] > 0):
            print("Accuracy for sentence ",i,": ",correct_senses[i]/total_senses[i])
        else:
            print("Accuracy for sentence ",i,": ",0)
# evaluate the accuracy of the algorithm in respect to all the sentences
print("Correct senses: ",sum(correct_senses))
print("Total senses: ",sum(total_senses))
print("Accuracy for all sentences: ",sum(correct_senses)/sum(total_senses))





[with stopwords]
Correct senses:  115
Total senses:  359
Accuracy for all sentences:  0.3203342618384401

[no stopwords]
Correct senses:  77
Total senses:  244
Accuracy for all sentences:  0.3155737704918033


 Randomize the selection of the 50 sentences and the selection of the term to be 
disambiguate, and return the average accuracy over (for example) 10 
program executions

In [80]:
import numpy as np

correct_senses= []
total_senses = []

for i in range(10):
    correct_senses_i, total_senses_i=  runTest(cleanse=True,randomExtract=True,debug=False)
    correct_senses.append(  correct_senses_i)
    total_senses.append(total_senses_i)
    

correct_senses_sum = 0
total_senses_sum   = 0
accuracies = np.zeros(len(correct_senses))
for i in range(len(correct_senses)):
    correct_senses_sum += sum(correct_senses[i])
    total_senses_sum  += sum(total_senses[i])
    # insert a value in the array accuracies
    accuracies[i]= sum(correct_senses[i])/sum(total_senses[i])

print("Average accuracy  {:.3f}+-{:.2f} over {} iterations: ".format(accuracies.mean(),
accuracies.std(), len(accuracies)))



Average accuracy  0.162+-0.12 over 10 iterations: 


In [81]:
import numpy as np

correct_senses= []
total_senses = []

for i in range(10):
    correct_senses_i, total_senses_i=  runTest(cleanse=False,randomExtract=True,debug=False)
    correct_senses.append(  correct_senses_i)
    total_senses.append(total_senses_i)
    

correct_senses_sum = 0
total_senses_sum   = 0
accuracies = np.zeros(len(correct_senses))
for i in range(len(correct_senses)):
    correct_senses_sum += sum(correct_senses[i])
    total_senses_sum  += sum(total_senses[i])
    # insert a value in the array accuracies
    accuracies[i]= sum(correct_senses[i])/sum(total_senses[i])

print("Average accuracy  {:.3f}+-{:.2f} over {} iterations: ".format(accuracies.mean(),
accuracies.std(), len(accuracies)))



Average accuracy  0.303+-0.03 over 10 iterations: 


# Summing up
The Lesk algorithm is a popular word sense disambiguation algorithm that uses the context of a word to determine its correct meaning. It relies on the overlap of word definitions in a given context to make sense of ambiguous words.
Function words are words that have little or no lexical meaning on their own but play a crucial role in the grammatical structure of a sentence. 

The impact of function words on the evaluation of the Lesk algorithm:
- **Contextual Significance**: Function words often carry important contextual information. They help establish relationships between other words in a sentence and provide clues about the syntactic structure. Removing function words may lead to a loss of this contextual information, which can negatively affect the accuracy of the algorithm.
- **Sense Discrimination**: Function words are typically highly frequent and appear in various contexts. By including function words in the evaluation, the algorithm can take into account the surrounding function words that may disambiguate the sense of the target word. 
- **Word Co-occurrence**: Function words often co-occur with content words (such as nouns, verbs, and adjectives) in specific patterns. By considering the function words along with the content words, the Lesk algorithm can leverage the co-occurrence patterns to disambiguate word senses more accurately. Eliminating function words might disrupt these patterns and result in suboptimal disambiguation results.

While eliminating function words might reduce noise in some cases, it can also hinder the algorithm's ability to capture the nuances of word sense disambiguation. By retaining function words, the Lesk algorithm can leverage their contributions to achieve better results.
**In this case the accouracy improved by around 87%.** (mean over 10 cycles)


 Function words do not necessarily impact the performance of the Lesk algorithm directly. In fact, their inclusion can often improve the algorithm's results by providing a larger signature and increasing the potential for overlaps.

### API TESTING 

In [ ]:
sentences = get_sentences_from_semcor()

for s in sentences[0][1][1:4]:
    if isinstance(s, nltk.tree.Tree): 
        print(s.label())
        for word in s:
            print(word)

[(['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', 'Atlanta', "'s", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], [['The'], Tree(Lemma('group.n.01.group'), [Tree('NE', ['Fulton', 'County', 'Grand', 'Jury'])]), Tree(Lemma('state.v.01.say'), ['said']), Tree(Lemma('friday.n.01.Friday'), ['Friday']), ['an'], Tree(Lemma('probe.n.01.investigation'), ['investigation']), ['of'], Tree(Lemma('atlanta.n.01.Atlanta'), ['Atlanta']), ["'s"], Tree(Lemma('late.s.03.recent'), ['recent']), Tree(Lemma('primary.n.01.primary_election'), ['primary', 'election']), Tree(Lemma('produce.v.04.produce'), ['produced']), ['``'], ['no'], Tree(Lemma('evidence.n.01.evidence'), ['evidence']), ["''"], ['that'], ['any'], Tree(Lemma('abnormality.n.04.irregularity'), ['irregularities']), Tree(Lemma('happen.v.01.take_place'), ['took', 'place']), ['.']])]
Lemma('group.n.01.group')
(NE Fulton County 

In [ ]:
# retrieve sentence sense from semcor
def get_sentence_sense_from_semcor(sentence):
    for i in range(len(semcor.tagged_sents())):
        if(semcor.sents()[i] == sentence):
            return semcor.tagged_sents()[i]



def get_nouns_from_sentence(sentenceData):
    nouns = []

    for pos, targetSense in enumerate(sentenceData[1]):
        if isinstance(targetSense, nltk.tree.Tree):
            lemma = targetSense.label()
            if(lemma):
                synset = None
                if(isinstance(lemma,str)):
                   synset = wn.synsets(lemma)
                else:
                    synset = lemma.synset()
                if synset and isinstance(synset,Synset) and  synset.pos() == 'n':
                    print(lemma,sentenceData[0][pos])
                    # print(synset)
                    print(synset.pos())
                    nouns.append(sentenceData[0][pos])
            
    return nouns

      
for i in zip(semcor.tagged_sents(tag="sem")[:2],semcor.sents()[:2]):
    print( [print(x) for x in i[0][1][0]])
    print(get_nouns_from_sentence((i[1],i[0])))
    # access the lemma of the word
    # get the type of i[0][1]
    # it is a nltk.tree.tree.Tree, so how to access the lemma?
    print(i[0][1].label().synset().name())
    # print(i[0][1].label().synset())
    
    print("\n")



Fulton
County
Grand
Jury
[None, None, None, None]
Lemma('group.n.01.group') Fulton
n
Lemma('friday.n.01.Friday') Grand
n
Lemma('probe.n.01.investigation') said
n
Lemma('atlanta.n.01.Atlanta') an
n
Lemma('primary.n.01.primary_election') Atlanta
n
Lemma('evidence.n.01.evidence') election
n
Lemma('abnormality.n.04.irregularity') evidence
n
['Fulton', 'Grand', 'said', 'an', 'Atlanta', 'election', 'evidence']
group.n.01


j
u
r
y
[None, None, None, None]
Lemma('jury.n.01.jury') jury
n
Lemma('term.n.02.term') term
n
Lemma('end.n.02.end') end
n
Lemma('presentment.n.01.presentment') presentments
n
Lemma('group.n.01.group') City
n
Lemma('mission.n.03.charge') had
n
Lemma('election.n.01.election') of
n
Lemma('praise.n.01.praise') deserves
n
Lemma('thanks.n.01.thanks') praise
n
Lemma('location.n.01.location') of
n
Lemma('manner.n.01.manner') Atlanta
n
Lemma('election.n.01.election') manner
n
['jury', 'term', 'end', 'presentments', 'City', 'had', 'of', 'deserves', 'praise', 'of', 'Atlanta', 'manne